# Assignment 3

In [1]:
import pandas as pd
import numpy as np

# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

<ipython-input-4-fb140fbf3b12>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_id'] = train_df['user_id'].map(user_dict)
<ipython-input-4-fb140fbf3b12>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remain_df['user_id'] = remain_df['user_id'].map(user_dict)


In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        0.0  2.0  0.0  4.0  0.0  4.0  4.0  0.0  0.0  2.0  ...  0.0  4.0  4.0   
 1        0.0  0.0  4.0  4.0  4.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 2        4.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 3        0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  4.0  ...  0.0  0.0  0.0   
 4        4.0  0.0  5.0  0.0  0.0  3.0  4.0  2.0  0.0  2.0  ...  0.0  2.0  0.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      4.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  4.0   
 296      4.0  0.0  4.0  0.0  0.0  4.0  3.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 297      4.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 298      0.0  0.0  0.0  3.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  5.0   
 299      0.0  0

In [6]:
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()

# Your implementation to predict the missing values
(Put all your implementation for your algorithm in the following cell only to handle the missing values; )

In [7]:
## Put all your implementation for your solutioin in this cell only to predict the missing values; 
## NOTE 1: DO NOT change anything in the rest of the cells in this framework, 
## otherwise the changes might cause errors and make your implementation invalid.

## Note 2: 
## The user-item rating matrix is imputed_train_ds, 
## and the missing values are those 0s in imputed_train_ds. 
## You are required to predict them by using the solution in the given report. 

## The following parameters are required in the given report, 
## which is named "Effective Missing Data Prediction for Collaborative Filtering", 
## and you will need to use them. But, please do not change their values. 
LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9
# finding the similarity for item - item
item_item_sim_matrix = np.zeros((imputed_train_ds.shape[1], imputed_train_ds.shape[1]))

for i, i_item_vec in enumerate(imputed_train_ds.T):
    for j, j_item_vec in enumerate(imputed_train_ds.T):

        i_mask = i_item_vec > 0
        j_mask = j_item_vec > 0

        corrated_index = np.intersect1d(np.where(i_mask), np.where(j_mask))
        if len(corrated_index) == 0:
            continue

        i_item_mean = np.sum(i_item_vec) / (np.sum(np.clip(i_item_vec, 0, 1)) + EPSILON)
        j_item_mean = np.sum(j_item_vec) / (np.sum(np.clip(j_item_vec, 0, 1)) + EPSILON)

        i_mean_item_sub = i_item_vec[corrated_index] - i_item_mean
        j_mean_item_sub = j_item_vec[corrated_index] - j_item_mean

        r_ui_sub_ri_sq = np.square(i_mean_item_sub)
        r_uj_sub_rj_sq = np.square(j_mean_item_sub)

        avg_sqrt_first_item = np.sqrt(np.sum(r_ui_sub_ri_sq))
        avg_sqrt_second_item = np.sqrt(np.sum(r_uj_sub_rj_sq))

        sim = np.sum(i_mean_item_sub * j_mean_item_sub) / (avg_sqrt_first_item * avg_sqrt_second_item + EPSILON)

        simlarity_weight = (min(len(corrated_index), DELTA) / DELTA) * sim
        
        
        item_item_sim_matrix[i][j] = simlarity_weight

        

#finding similarity for users


user_user_sim_matrix = np.zeros((imputed_train_ds.shape[0], imputed_train_ds.shape[0]))

for i, i_user_vec in enumerate(imputed_train_ds):
    for j, j_user_vec in enumerate(imputed_train_ds):

        i_mask = i_user_vec > 0
        j_mask = j_user_vec > 0

        corrated_index = np.intersect1d(np.where(i_mask), np.where(j_mask))
        if len(corrated_index) == 0:
            continue

        i_user_mean = np.sum(i_user_vec) / (np.sum(np.clip(i_user_vec, 0, 1)) + EPSILON)
        j_user_mean = np.sum(j_user_vec) / (np.sum(np.clip(j_user_vec, 0, 1)) + EPSILON)

        i_mean_user_sub = i_user_vec[corrated_index] - i_user_mean
        j_mean_user_sub = j_user_vec[corrated_index] - j_user_mean

        r_ui_sub_ri_sq = np.square(i_mean_user_sub)
        r_uj_sub_rj_sq = np.square(j_mean_user_sub)

        avg_sqrt_first_user = np.sqrt(np.sum(r_ui_sub_ri_sq))
        avg_sqrt_second_user = np.sqrt(np.sum(r_uj_sub_rj_sq))

        sim = np.sum(i_mean_user_sub * j_mean_user_sub) / (avg_sqrt_first_user * avg_sqrt_second_user + EPSILON)

        simlarity_weight = (min(len(corrated_index), GAMMA) / GAMMA) * sim
        
        user_user_sim_matrix[i][j] =  simlarity_weight

        
        
        
np_predictions = np.zeros((imputed_train_ds.shape[0], imputed_train_ds.shape[1]))


for (i, j), rating in np.ndenumerate(imputed_train_ds):
    
         
    #user-item

        #find the similar users
        sim_user_ids = np.argsort(user_user_sim_matrix[i])
        #similarity co-efficent of values of user 
        sim_value = []
        sim_user_ids_1 = []
        for m in range(len(user_user_sim_matrix[i][sim_user_ids])):

            if user_user_sim_matrix[i][sim_user_ids][m] > ITA and user_user_sim_matrix[i][sim_user_ids][m] < 1:

                sim_value.append(user_user_sim_matrix[i][sim_user_ids][m])
                sim_user_ids_1.append(sim_user_ids[m])
        sim_value = np.array(sim_value)
        sim_user_ids_1 = np.array(sim_user_ids_1)
    
    #item-user
        
        sim_item_ids = np.argsort(item_item_sim_matrix[j])
        sim_item_value =[]
        sim_item_ids_1=[]
        
        for n in range(len(item_item_sim_matrix[j][sim_item_ids])):
                      
            if item_item_sim_matrix[j][sim_item_ids][n] > THETA and item_item_sim_matrix[j][sim_item_ids][n] < 1:
                
                sim_item_value.append(item_item_sim_matrix[j][sim_item_ids][n])
                sim_item_ids_1.append(sim_item_ids[n])
        
        sim_item_value = np.array(sim_item_value)
        sim_item_ids_1 = np.array(sim_item_ids_1)
        
        if (sim_item_value.size > 0) and (sim_value.size == 0):

            
            sim_items = imputed_train_ds.T[sim_item_ids_1]
            item_mean = np.sum(imputed_train_ds.T[j]) / (np.sum(np.clip(imputed_train_ds.T[j], 0, 1))+EPSILON)
            sim_item_mean = np.sum(sim_items,axis = 1) / (np.sum(np.clip(sim_items, 0 ,1),axis = 1)+EPSILON)
            mask_rated_i = sim_items[:,i] >0
            
            sim_r_sum_mean = sim_item_value[mask_rated_i] * (sim_items[mask_rated_i, i] - sim_item_mean[mask_rated_i])

            np_predictions[i][j] = item_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_item_value[mask_rated_i]) + EPSILON)
            np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
            
            if imputed_train_ds[i][j] == 0:
                imputed_train_ds[i][j] = np_predictions[i][j]

            
        elif (sim_value.size > 0) and (sim_item_value.size == 0):

          
            sim_users = imputed_train_ds[sim_user_ids_1]
            
            user_mean = np.sum(imputed_train_ds[i]) / (np.sum(np.clip(imputed_train_ds[i], 0, 1))+EPSILON)
            sim_user_mean = np.sum(sim_users,axis = 1) / (np.sum(np.clip(sim_users, 0 ,1),axis = 1)+EPSILON)
            mask_rated_j = sim_users[:,j] >0

            sim_r_sum_mean = sim_value[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])

            np_predictions[i][j] = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_value[mask_rated_j]) + EPSILON)
            np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
            
            if imputed_train_ds[i][j] == 0:
                imputed_train_ds[i][j] = np_predictions[i][j]
            
            
        elif(sim_value.size > 0) and (sim_item_value.size > 0):

            sim_items = imputed_train_ds.T[sim_item_ids_1]
            item_mean = np.sum(imputed_train_ds.T[j]) / (np.sum(np.clip(imputed_train_ds.T[j], 0, 1))+EPSILON)
            sim_item_mean = np.sum(sim_items,axis = 1) / (np.sum(np.clip(sim_items, 0 ,1),axis = 1)+EPSILON)
            mask_rated_i = sim_items[:,i] >0
            
            sim_r_sum_mean = sim_item_value[mask_rated_i] * (sim_items[mask_rated_i, i] - sim_item_mean[mask_rated_i])

            item = item_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_item_value[mask_rated_i]) + EPSILON)
            
            
            
            sim_users = imputed_train_ds[sim_user_ids_1]
            user_mean = np.sum(imputed_train_ds[i]) / (np.sum(np.clip(imputed_train_ds[i], 0, 1))+EPSILON)
            sim_user_mean = np.sum(sim_users,axis = 1) / (np.sum(np.clip(sim_users, 0 ,1),axis = 1)+EPSILON)
            mask_rated_j = sim_users[:,j] >0

            sim_r_sum_mean = sim_value[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])

            user = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_value[mask_rated_j]) + EPSILON)
            
            np_predictions[i][j] = ((LAMBDA)*(user) )+ ((1 - LAMBDA)*(item))
            
            np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
            
            if imputed_train_ds[i][j] == 0:
                imputed_train_ds[i][j] = np_predictions[i][j]
        else:
            np_predictions[i][j] = 0
            if imputed_train_ds[i][j] == 0:
                imputed_train_ds[i][j] = np_predictions[i][j]
                
            



# Evaluation

### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [8]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,3.829127,2.000000,4.209420,4.000000,3.439398,4.000000,4.000000,3.686601,3.293528,2.000000,...,3.490373,4.000000,4.000000,3.738610,3.000000,3.000000,3.208256,3.735092,3.447461,3.223310
1,4.091690,4.004791,4.000000,4.000000,4.000000,3.847684,4.052298,4.000000,3.828374,3.946646,...,3.831194,3.761851,4.050723,3.933634,3.695716,3.656909,3.659541,4.033253,3.648585,3.790726
2,4.000000,3.540059,4.056860,2.000000,4.025909,3.785061,3.987944,3.883311,3.670460,3.881337,...,3.783769,3.679259,3.767874,3.871586,3.622118,3.619349,3.469084,3.805717,3.592746,3.643307
3,4.040227,4.137668,4.072213,3.959556,5.000000,3.801418,4.002911,3.770500,3.688766,4.000000,...,3.764434,3.700436,3.958327,3.879275,3.649267,3.686861,3.478475,3.956503,3.606777,3.805198
4,4.000000,3.215861,5.000000,3.547428,3.330157,3.000000,4.000000,2.000000,3.396593,2.000000,...,3.216427,2.000000,3.656549,3.539524,3.020071,2.857111,3.067230,4.000000,3.272274,3.056421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.000000,3.655659,4.676927,2.623083,3.390411,2.505079,4.238273,4.000000,3.752715,3.914452,...,3.782010,3.226528,4.000000,3.791034,2.729840,3.561633,2.928846,3.978315,3.413422,3.165319
296,4.000000,3.575754,4.000000,3.814560,3.497203,4.000000,3.000000,4.000000,3.689859,3.783807,...,3.759570,3.696846,3.773099,3.769784,3.639692,3.775224,3.666880,3.744009,3.678147,3.692321
297,4.000000,3.228612,3.000000,3.748918,3.650731,3.945513,3.751572,3.444896,3.362871,3.604954,...,3.513165,3.378540,3.900349,3.663103,3.230792,3.442364,3.246073,3.561631,3.494416,3.527495
298,4.154619,2.796488,3.331390,3.000000,3.101568,3.067835,3.256615,4.000000,3.314203,3.386628,...,1.908426,3.130069,5.000000,3.387452,3.169766,3.000000,3.058678,3.300555,3.231282,3.083397


In [9]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair od users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

array([[ 0.04814179, -0.29222158, -0.16368312, ...,  0.08154789,
         0.39953132, -0.17553639],
       [-0.13176887,  0.03146049, -0.17023023, ...,  0.37312952,
         0.39567131,  0.28478696],
       [ 0.04593563,  0.0839935 , -0.16844836, ..., -0.22212957,
        -0.04784197,  0.15393656],
       ...,
       [ 0.20204093,  0.38926563, -0.20212429, ..., -0.06731283,
        -0.14874741,  0.41765361],
       [ 0.1700147 ,  0.38987035, -0.09248602, ...,  0.02744657,
         0.15250432, -0.06140499],
       [ 0.12481482,  0.28484048,  0.24550622, ...,  0.48839251,
         0.45195605,  0.1431662 ]])

## Predict Ratings of Testing Set

In [10]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred


array([[0.        , 0.        , 0.        , ..., 0.        , 3.35387411,
        0.        ],
       [3.53987511, 0.        , 3.46931344, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 3.40495677, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 2.90881134, ..., 0.        , 2.43176473,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 3.33789196, 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Compute MAE and RMSE

In [11]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

MAE: 0.7671841127059978, RMSE: 0.9801677612584818


In [12]:
0.8000032870645626 - MAE

0.03281917435856474

In [13]:
1.0251400801628212 - RMSE

0.04497231890433939